In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from pathlib import Path

from src.models.association_rules import AssociationRuleModel
from src.models.collaborative_filtering import CollaborativeFilteringModel
from src.models.hybrid_recommender import HybridRecommender

# FORCE_RETRAIN_AR=True


In [2]:
# ====================== Config ======================
MODELS_DIR = Path("../models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

AR_PATH = MODELS_DIR / "association_rule_model.pkl"
CF_PATH = MODELS_DIR / "cf_model_als.pkl"

# 默认不重训：优先加载已有checkpoint
RETRAIN_AR_IF_MISSING = True
RETRAIN_CF_IF_MISSING = True

# 可选：如果你想手动强制重训，改成 True
FORCE_RETRAIN_AR = False
FORCE_RETRAIN_CF = False

# Hybrid权重
AR_WEIGHT = 0.4
CF_WEIGHT = 0.6

# 可选：是否自动搜索更优权重
TUNE_HYBRID_WEIGHTS = True


In [3]:
# ====================== Load Data ======================
implicit_ratings = pd.read_csv("../data/processed/implicit_ratings.csv")
prior = pd.read_csv("../data/processed/prior_filtered.csv")
products = pd.read_csv("../data/raw/products.csv")

print(f"implicit_ratings: {len(implicit_ratings):,} rows")
print(f"prior: {len(prior):,} rows")
print(f"products: {len(products):,} rows")
print(f"users: {implicit_ratings['user_id'].nunique():,}, items: {implicit_ratings['product_id'].nunique():,}")

implicit_ratings: 12,083,736 rows
prior: 30,718,986 rows
products: 49,688 rows
users: 162,381, items: 35,922


In [4]:
# ====================== Per-user Split (for offline evaluation) ======================

def per_user_split_df(df_in, test_ratio=0.2, seed=42):
    rng_local = np.random.RandomState(seed)
    tr_list, te_list = [], []
    for _, g in df_in.groupby("user_id"):
        n = len(g)
        k = max(1, int(np.floor(test_ratio * n)))
        idx = rng_local.choice(g.index, size=k, replace=False)
        te_list.append(g.loc[idx])
        tr_list.append(g.drop(idx))
    tr = pd.concat(tr_list).reset_index(drop=True)
    te = pd.concat(te_list).reset_index(drop=True)
    return tr, te

cnt = implicit_ratings.groupby("user_id").size()
eligible_users = cnt[cnt >= 5].index
cf_df = implicit_ratings[implicit_ratings["user_id"].isin(eligible_users)].copy()

train_ratings, test_ratings = per_user_split_df(cf_df, test_ratio=0.2, seed=42)

print("Train rows:", len(train_ratings), "Test rows:", len(test_ratings))
print("Train users:", train_ratings["user_id"].nunique(), "Test users:", test_ratings["user_id"].nunique())
print("User overlap:", test_ratings["user_id"].isin(train_ratings["user_id"]).mean())
print("Item overlap:", test_ratings["product_id"].isin(train_ratings["product_id"]).mean())


Train rows: 9727302 Test rows: 2351910
Train users: 160932 Test users: 160932
User overlap: 1.0
Item overlap: 1.0


In [5]:
# ====================== Load/Train CF Model ======================
if CF_PATH.exists() and not FORCE_RETRAIN_CF:
    print(f"Loading CF model from {CF_PATH}")
    cf_model = CollaborativeFilteringModel.load(str(CF_PATH))
else:
    if not RETRAIN_CF_IF_MISSING and not FORCE_RETRAIN_CF:
        raise FileNotFoundError(f"CF checkpoint missing: {CF_PATH}")

    print("Training CF model from scratch...")
    cf_model = CollaborativeFilteringModel(
        method='als',
        n_factors=64,
        regularization=0.05,
        iterations=20,
        alpha=20.0,
        fallback_on_invalid=False,
    )
    cf_model.fit(train_ratings)
    cf_model.save(str(CF_PATH))
    print(f"Saved CF model -> {CF_PATH}")

# 即使是加载模型，也在当前test上做一次离线评估
metrics_cf = cf_model.evaluate(test_ratings)
print("CF core metrics:", {k: metrics_cf.get(k) for k in ["rmse","mae","correlation","coverage"]})


Loading CF model from ../models/cf_model_als.pkl


INFO:src.models.collaborative_filtering:Model loaded from ../models/cf_model_als.pkl
INFO:src.models.collaborative_filtering:Evaluating model...
INFO:src.models.collaborative_filtering:Eval diagnostics: cold_start_user_rate=0.00%, cold_start_item_rate=0.00%, in_matrix_rate=100.00%
INFO:src.models.collaborative_filtering:Eval diagnostics: dropped_empty_items=0
INFO:src.models.collaborative_filtering:RMSE: 0.3026
INFO:src.models.collaborative_filtering:MAE: 0.2161
INFO:src.models.collaborative_filtering:Correlation: 0.3204
INFO:src.models.collaborative_filtering:Spearman Correlation: 0.2144
INFO:src.models.collaborative_filtering:Coverage (all test rows): 100.00%
INFO:src.models.collaborative_filtering:Coverage (in-matrix rows only): 100.00%
INFO:src.models.collaborative_filtering:Fallback rate (in-matrix): 0.00% | Non-finite drop rate (in-matrix): 0.00%
INFO:src.models.collaborative_filtering:Predict status counts: {'ok': 2351910}


CF core metrics: {'rmse': 0.3026467583740583, 'mae': 0.21605173041505613, 'correlation': 0.3203880912527413, 'coverage': 1.0}


In [6]:
# ====================== Load/Train AR Model ======================
if AR_PATH.exists() and not FORCE_RETRAIN_AR:
    print(f"Loading AR model from {AR_PATH}")
    ar_model = AssociationRuleModel.load(str(AR_PATH))
else:
    if not RETRAIN_AR_IF_MISSING and not FORCE_RETRAIN_AR:
        raise FileNotFoundError(f"AR checkpoint missing: {AR_PATH}")

    print("Training AR model from scratch...")
    ar_model = AssociationRuleModel(
        min_support=0.005,
        min_confidence=0.1,
        min_lift=2.0,
        max_len=2,
    )
    ar_model.fit(prior, top_k_products=1000)
    ar_model.save(str(AR_PATH))
    print(f"Saved AR model -> {AR_PATH}")

print(f"AR rules: {len(ar_model.rules):,}")


INFO:src.models.association_rules:Model loaded from ../models/association_rule_model.pkl


Loading AR model from ../models/association_rule_model.pkl
AR rules: 277


In [7]:
def _get_user_history(self, user_id: int, top_k: int = 30):
    """Get user's interacted products from CF training matrix (by row strength)."""
    if (
        getattr(self.cf_model, "user_encoder", None) is None
        or getattr(self.cf_model, "user_item_matrix", None) is None
        or getattr(self.cf_model, "product_decoder", None) is None
    ):
        return []

    user_keys = self.cf_model.user_encoder
    if len(user_keys) == 0:
        return []

    # best-effort cast to encoder key type
    try:
        key_type = type(next(iter(user_keys.keys())))
        user_key = key_type(user_id)
    except Exception:
        user_key = user_id

    if user_key not in user_keys:
        return []

    user_idx = int(user_keys[user_key])
    row = self.cf_model.user_item_matrix.tocsr()[user_idx]

    if row.nnz == 0:
        return []

    # Sort by interaction strength descending
    order = np.argsort(row.data)[::-1]
    item_indices = row.indices[order]
    if top_k is not None and top_k > 0:
        item_indices = item_indices[:top_k]

    history = []
    for col_idx in item_indices:
        pid = self.cf_model.product_decoder.get(int(col_idx))
        if pid is not None:
            history.append(pid)

    return history


In [8]:
def recommend_for_user(
    self,
    user_id: int,
    top_n: int = 10,
    exclude_purchased: bool = True,) :
    """
    Hybrid user recommendations:
    - CF personalized candidates
    - AR candidates expanded from user's historical products
    - Weighted fusion of normalized scores
    """
    # 1) CF candidates
    cf_recs = self.cf_model.get_recommendations(
        user_id,
        top_n=top_n * 5,
        exclude_purchased=exclude_purchased,
    )
    cf_norm = self._normalize_scores(cf_recs)

    # 2) AR candidates from user history
    user_history = self._get_user_history(user_id, top_k=30)
    ar_agg = {}  # pid -> best lift

    for seed_pid in user_history:
        ar_recs = self.ar_model.get_recommendations(int(seed_pid), top_n=top_n * 3, only_pairs=True)
        for rec_pid, lift in ar_recs:
            if rec_pid == seed_pid:
                continue
            score = float(lift)
            if (rec_pid not in ar_agg) or (score > ar_agg[rec_pid]):
                ar_agg[rec_pid] = score

    ar_norm = self._normalize_scores(list(ar_agg.items()))

    # 3) Merge candidate pool
    candidate_ids = set(cf_norm.keys()) | set(ar_norm.keys())
    if exclude_purchased:
        purchased = set(user_history)
        candidate_ids = {pid for pid in candidate_ids if pid not in purchased}

    if not candidate_ids:
        logger.warning(f"No hybrid recommendations for user {user_id}")
        return []

    # 4) Weighted fusion
    hybrid_scores = []
    for pid in candidate_ids:
        ar_s = ar_norm.get(pid, 0.0)
        cf_s = cf_norm.get(pid, 0.0)
        score = self.ar_weight * ar_s + self.cf_weight * cf_s

        if ar_s > 0 and cf_s > 0:
            source = "HYBRID"
        elif cf_s > 0:
            source = "CF"
        else:
            source = "AR"

        hybrid_scores.append((pid, float(score), source))

    hybrid_scores.sort(key=lambda x: x[1], reverse=True)
    return hybrid_scores[:top_n]


In [9]:
# ====================== Build Hybrid ======================
hybrid = HybridRecommender(
    ar_model=ar_model,
    cf_model=cf_model,
    ar_weight=AR_WEIGHT,
    cf_weight=CF_WEIGHT,
)

# 建一个 product_id -> product_name 的字典，打印更快
product_name_map = products.set_index("product_id")["product_name"].to_dict()

def name_of(pid):
    try:
        return product_name_map.get(int(pid), f"Product {pid}")
    except Exception:
        return f"Product {pid}"

def display_recs_with_names(recs, title, top_n=5):
    print(title)
    if not recs:
        print("  (no recommendations)")
        return
    for i, (pid, score, source) in enumerate(recs[:top_n], 1):
        print(f"  {i}. {name_of(pid)} | id={pid} | score={score:.4f} | source={source}")

sample_user = np.random.choice(train_ratings["user_id"].unique())
recs_user = hybrid.recommend_for_user(sample_user, top_n=10)
print("sample user:", sample_user)
display_recs_with_names(recs_user, "top-5 user recs（For you）:", top_n=5)

sample_basket = train_ratings[train_ratings["user_id"] == sample_user]["product_id"].head(3).tolist()
print("sample basket product_ids(购物车页 “Frequently bought together):", sample_basket)
recs_basket = hybrid.recommend_for_basket(sample_basket, user_id=sample_user, top_n=10)
display_recs_with_names(recs_basket, "top-5 basket recs:", top_n=5)


sample user: 170183
top-5 user recs（For you）:
  1. Organic Cucumber | id=30391 | score=0.7330 | source=HYBRID
  2. Organic Baby Spinach | id=21903 | score=0.6554 | source=HYBRID
  3. Organic Italian Parsley Bunch | id=34126 | score=0.6456 | source=HYBRID
  4. Organic Yellow Onion | id=22935 | score=0.6172 | source=HYBRID
  5. Half & Half | id=27086 | score=0.5896 | source=CF
sample basket product_ids(购物车页 “Frequently bought together): [1463, 1658, 1947]
top-5 basket recs:
  1. Organic Baby Spinach | id=21903 | score=0.6000 | source=CF
  2. Half & Half | id=27086 | score=0.5840 | source=CF
  3. Organic Cucumber | id=30391 | score=0.3183 | source=CF
  4. Organic Reduced Fat 2% Milk | id=5785 | score=0.2331 | source=CF
  5. Spring Water | id=19660 | score=0.2202 | source=CF


In [10]:
# ====================== Evaluate Hybrid (user-side Hit@K) ======================

def evaluate_user_hitk(model_hybrid, test_df, k=10, max_users=5000, seed=42):
    rng_local = np.random.RandomState(seed)
    user_to_items = test_df.groupby("user_id")["product_id"].apply(set)
    users = np.array(list(user_to_items.index))
    if len(users) > max_users:
        users = rng_local.choice(users, size=max_users, replace=False)

    hits = 0
    precisions = []
    recalls = []

    for u in users:
        gt = user_to_items[u]
        recs = model_hybrid.recommend_for_user(u, top_n=k, exclude_purchased=True)
        rec_items = [pid for pid, _, _ in recs]

        if not rec_items:
            precisions.append(0.0)
            recalls.append(0.0)
            continue

        inter = len(set(rec_items) & gt)
        if inter > 0:
            hits += 1
        precisions.append(inter / k)
        recalls.append(inter / max(1, len(gt)))

    n = len(users)
    return {
        "users_evaluated": int(n),
        "hit_rate@k": hits / n if n else 0.0,
        "precision@k": float(np.mean(precisions)) if precisions else 0.0,
        "recall@k": float(np.mean(recalls)) if recalls else 0.0,
    }

hybrid_user_eval = evaluate_user_hitk(hybrid, test_ratings, k=10, max_users=5000, seed=42)
print("Hybrid user eval:", hybrid_user_eval)


Hybrid user eval: {'users_evaluated': 5000, 'hit_rate@k': 0.443, 'precision@k': 0.07374, 'recall@k': 0.05914282647650266}


In [11]:
# ====================== Optional: Tune Hybrid Weights ======================
if TUNE_HYBRID_WEIGHTS:
    weight_grid = [
        (0.2, 0.8),
        (0.3, 0.7),
        (0.4, 0.6),
        (0.5, 0.5),
        (0.6, 0.4),
    ]

    tune_rows = []
    for ar_w, cf_w in weight_grid:
        h = HybridRecommender(
            ar_model=ar_model,
            cf_model=cf_model,
            ar_weight=ar_w,
            cf_weight=cf_w,
        )
        m = evaluate_user_hitk(h, test_ratings, k=10, max_users=3000, seed=42)
        tune_rows.append({
            "ar_weight": ar_w,
            "cf_weight": cf_w,
            **m,
        })

    tune_df = pd.DataFrame(tune_rows).sort_values("hit_rate@k", ascending=False).reset_index(drop=True)
    print(tune_df)

    best = tune_df.iloc[0]
    print("Best hybrid weights:", {"ar_weight": best["ar_weight"], "cf_weight": best["cf_weight"]})
    """
    hybrid = HybridRecommender(
        ar_model=ar_model,
        cf_model=cf_model,
        ar_weight=float(best["ar_weight"]),
        cf_weight=float(best["cf_weight"]),
    )
    """
    hybrid = HybridRecommender(
    ar_model=ar_model,
    cf_model=cf_model,
    ar_weight=AR_WEIGHT,
    cf_weight=CF_WEIGHT,
    enable_fallback=True,                 
    cf_user_candidate_multiplier=20,
    ar_user_seed_multiplier=5,
    ar_basket_candidate_multiplier=4,
    cf_basket_candidate_multiplier=2,
    ar_basket_require_full_match=False,
    ar_basket_min_overlap=1,
    ar_basket_overlap_weighted=True,
)


   ar_weight  cf_weight  users_evaluated  hit_rate@k  precision@k  recall@k
0        0.6        0.4             3000    0.435000     0.071833  0.059336
1        0.3        0.7             3000    0.432667     0.072333  0.059372
2        0.4        0.6             3000    0.432333     0.071733  0.058577
3        0.5        0.5             3000    0.432000     0.071433  0.058364
4        0.2        0.8             3000    0.430000     0.071933  0.059372
Best hybrid weights: {'ar_weight': np.float64(0.6), 'cf_weight': np.float64(0.4)}


In [12]:
cold_user = int(train_ratings["user_id"].max()) + 10_000_000
cold_recs = hybrid.recommend_for_user(cold_user, top_n=10, exclude_purchased=True)
print(cold_recs[:5])   # 这里 source 应该能看到 FALLBACK_POPULAR


INFO:src.models.hybrid_recommender:User 10206209 fallback recommendations returned: 10


[(np.int64(24852), 1.0, 'FALLBACK_POPULAR'), (np.int64(13176), 0.7229090653523066, 'FALLBACK_POPULAR'), (np.int64(21137), 0.47466658454831134, 'FALLBACK_POPULAR'), (np.int64(21903), 0.42770689916620364, 'FALLBACK_POPULAR'), (np.int64(47209), 0.258715910259973, 'FALLBACK_POPULAR')]


In [13]:
# ====================== Save Hybrid-Context Checkpoints (optional names) ======================
# 这里只是另存一份，避免覆盖你主流程checkpoint
CF_HYBRID_PATH = MODELS_DIR / "cf_model_hybrid_als.pkl"
AR_HYBRID_PATH = MODELS_DIR / "association_rule_model_hybrid.pkl"

cf_model.save(str(CF_HYBRID_PATH))
ar_model.save(str(AR_HYBRID_PATH))

print("Saved:")
print(CF_HYBRID_PATH)
print(AR_HYBRID_PATH)


INFO:src.models.collaborative_filtering:Model saved to ../models/cf_model_hybrid_als.pkl
INFO:src.models.association_rules:Model saved to ../models/association_rule_model_hybrid.pkl


Saved:
../models/cf_model_hybrid_als.pkl
../models/association_rule_model_hybrid.pkl


In [14]:
# ====================== Evaluate Hybrid at Multiple K ======================
def evaluate_user_hitk(model_hybrid, test_df, k=10, max_users=5000, seed=42):
    rng_local = np.random.RandomState(seed)
    user_to_items = test_df.groupby("user_id")["product_id"].apply(set)
    users = np.array(list(user_to_items.index))
    if len(users) > max_users:
        users = rng_local.choice(users, size=max_users, replace=False)

    hits = 0
    precisions = []
    recalls = []

    for u in users:
        gt = user_to_items[u]
        recs = model_hybrid.recommend_for_user(u, top_n=k, exclude_purchased=True)
        rec_items = [pid for pid, _, _ in recs]

        if not rec_items:
            precisions.append(0.0)
            recalls.append(0.0)
            continue

        inter = len(set(rec_items) & gt)
        if inter > 0:
            hits += 1
        precisions.append(inter / k)
        recalls.append(inter / max(1, len(gt)))

    n = len(users)
    return {
        "users_evaluated": int(n),
        "hit_rate": hits / n if n else 0.0,
        "precision": float(np.mean(precisions)) if precisions else 0.0,
        "recall": float(np.mean(recalls)) if recalls else 0.0,
    }

K_LIST = [5, 10, 20]
rows = []

for k in K_LIST:
    m = evaluate_user_hitk(hybrid, test_ratings, k=k, max_users=5000, seed=42)
    rows.append({
        "k": k,
        "users_evaluated": int(m["users_evaluated"]),
        "hit_rate": float(m["hit_rate"]),
        "precision": float(m["precision"]),
        "recall": float(m["recall"]),
    })

eval_k_df = pd.DataFrame(rows).sort_values("k").reset_index(drop=True)

# optional: nicer display
eval_k_df["hit_rate"] = eval_k_df["hit_rate"].map(lambda x: f"{x:.4f}")
eval_k_df["precision"] = eval_k_df["precision"].map(lambda x: f"{x:.4f}")
eval_k_df["recall"] = eval_k_df["recall"].map(lambda x: f"{x:.4f}")

eval_k_df



,k,users_evaluated,hit_rate,precision,recall
0,5,5000,0.3276,0.0902,0.0371
1,10,5000,0.4430,0.0737,0.0591
2,20,5000,0.5522,0.0582,0.0907


In [15]:
# ====================== MIXED WARM/COLD EVALUATION (INDUSTRY-STYLE TOLERANCE) ======================

def _prh_at_k(rec_items, gt_items, k):
    rec_set = set(rec_items[:k])
    gt_set = set(gt_items)
    inter = len(rec_set & gt_set)
    hit = 1.0 if inter > 0 else 0.0
    precision = inter / max(1, k)
    recall = inter / max(1, len(gt_set))
    return hit, precision, recall

def evaluate_mixed_cold_start(
    model_hybrid,
    test_df,
    cold_user_ratio=0.10,
    k=10,
    basket_seed_k=3,
    max_users=3000,
    seed=42,
):
    rng = np.random.RandomState(seed)
    user_to_items = test_df.groupby("user_id")["product_id"].apply(list)
    users = np.array(list(user_to_items.index))
    if len(users) > max_users:
        users = rng.choice(users, size=max_users, replace=False)

    n_users = len(users)
    n_cold = int(round(n_users * cold_user_ratio))
    cold_users = set(rng.choice(users, size=n_cold, replace=False)) if n_cold > 0 else set()

    user_hits, user_precs, user_recalls = [], [], []
    basket_hits, basket_precs, basket_recalls = [], [], []
    serve_hits, serve_precs, serve_recalls = [], [], []

    for idx, u in enumerate(users):
        items = list(dict.fromkeys(user_to_items[u]))  # dedup keep order
        if len(items) == 0:
            continue

        seed_items = items[:min(basket_seed_k, len(items))]
        gt_items = items[min(basket_seed_k, len(items)):]
        if len(gt_items) == 0:
            gt_items = items

        is_cold = u in cold_users
        if is_cold:
            cold_uid = int(u) + 10_000_000 + idx
            user_recs = model_hybrid.recommend_for_user(cold_uid, top_n=k, exclude_purchased=True)
            basket_recs = model_hybrid.recommend_for_basket(seed_items, user_id=None, top_n=k)
        else:
            user_recs = model_hybrid.recommend_for_user(int(u), top_n=k, exclude_purchased=True)
            basket_recs = model_hybrid.recommend_for_basket(seed_items, user_id=int(u), top_n=k)

        user_items = [pid for pid, _, _ in user_recs]
        basket_items = [pid for pid, _, _ in basket_recs]
        served_items = user_items if len(user_items) > 0 else basket_items

        h, p, r = _prh_at_k(user_items, gt_items, k)
        user_hits.append(h); user_precs.append(p); user_recalls.append(r)

        h, p, r = _prh_at_k(basket_items, gt_items, k)
        basket_hits.append(h); basket_precs.append(p); basket_recalls.append(r)

        h, p, r = _prh_at_k(served_items, gt_items, k)
        serve_hits.append(h); serve_precs.append(p); serve_recalls.append(r)

    def _mean(x):
        return float(np.mean(x)) if len(x) > 0 else 0.0

    return {
        "users_evaluated": int(len(user_hits)),
        "cold_user_ratio": float(cold_user_ratio),
        "user_hit_rate@k": _mean(user_hits),
        "user_precision@k": _mean(user_precs),
        "user_recall@k": _mean(user_recalls),
        "basket_hit_rate@k": _mean(basket_hits),
        "basket_precision@k": _mean(basket_precs),
        "basket_recall@k": _mean(basket_recalls),
        "served_hit_rate@k": _mean(serve_hits),
        "served_precision@k": _mean(serve_precs),
        "served_recall@k": _mean(serve_recalls),
    }

ratios = [0.10, 0.15, 0.20]
rows = []
for r in ratios:
    m = evaluate_mixed_cold_start(
        hybrid,
        test_ratings,
        cold_user_ratio=r,
        k=10,
        basket_seed_k=3,
        max_users=3000,
        seed=42,
    )
    rows.append(m)

mixed_cs_df = pd.DataFrame(rows)
print(mixed_cs_df[[
    "cold_user_ratio", "users_evaluated",
    "user_hit_rate@k", "basket_hit_rate@k", "served_hit_rate@k",
    "user_precision@k", "basket_precision@k", "served_precision@k",
    "user_recall@k", "basket_recall@k", "served_recall@k",
]])


INFO:src.models.hybrid_recommender:User 10183713 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10106533 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10140038 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10140833 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10105742 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10146672 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10196451 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10202455 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10007175 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10197184 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 10191665 fallback recommendations returned: 10
INFO:src.models.hybrid_recommender:User 101

   cold_user_ratio  users_evaluated  user_hit_rate@k  basket_hit_rate@k  \
0             0.10             3000         0.362667           0.335667   
1             0.15             3000         0.355333           0.329667   
2             0.20             3000         0.353000           0.329000   

   served_hit_rate@k  user_precision@k  basket_precision@k  \
0           0.362667          0.057033            0.052567   
1           0.355333          0.055767            0.051600   
2           0.353000          0.054833            0.051067   

   served_precision@k  user_recall@k  basket_recall@k  served_recall@k  
0            0.057033       0.058846         0.044625         0.058846  
1            0.055767       0.057030         0.043474         0.057030  
2            0.054833       0.056038         0.043378         0.056038  


In [18]:
# ====================== CF user-holdout + fallback test ======================
SEED = 42
TEST_USER_RATIO = 0.20
K = 10
MAX_EVAL_USERS = 5000

# 如果训练太慢，先开 True 做快速验证
FAST_MODE = False
TRAIN_USER_CAP = 80000

rng = np.random.RandomState(SEED)

# 1) user-holdout split (natural cold-start)
all_users = implicit_ratings["user_id"].unique()
n_test_users = int(len(all_users) * TEST_USER_RATIO)
test_users = set(rng.choice(all_users, size=n_test_users, replace=False))
train_users = set(all_users) - test_users

train_holdout = implicit_ratings[implicit_ratings["user_id"].isin(train_users)].copy()
test_holdout = implicit_ratings[implicit_ratings["user_id"].isin(test_users)].copy()

print("Train rows:", len(train_holdout), "Test rows:", len(test_holdout))
print("Train users:", train_holdout["user_id"].nunique(), "Test users:", test_holdout["user_id"].nunique())
print("User overlap:", test_holdout["user_id"].isin(train_holdout["user_id"]).mean())  # 期望接近 0

fit_df = train_holdout
if FAST_MODE:
    tr_users = np.array(list(train_users))
    sub_users = set(rng.choice(tr_users, size=min(TRAIN_USER_CAP, len(tr_users)), replace=False))
    fit_df = train_holdout[train_holdout["user_id"].isin(sub_users)].copy()
    print("FAST_MODE on | fit rows:", len(fit_df), "| fit users:", fit_df["user_id"].nunique())

# 2) train CF on holdout-train
cf_holdout = CollaborativeFilteringModel(
    method="als",
    n_factors=48,
    regularization=0.05,
    iterations=10,
    alpha=20.0,
    fallback_on_invalid=False,
)
cf_holdout.fit(fit_df)

# pair-level coverage (你原来那个coverage定义)
m_cf = cf_holdout.evaluate(test_holdout)
print("CF pair-level metrics:", {
    "rmse": m_cf.get("rmse"),
    "mae": m_cf.get("mae"),
    "correlation": m_cf.get("correlation"),
    "coverage": m_cf.get("coverage"),
    "coverage_in_matrix": m_cf.get("coverage_in_matrix"),
    "cold_start_user_rate": m_cf.get("cold_start_user_rate"),
    "cold_start_item_rate": m_cf.get("cold_start_item_rate"),
})

# 3) build popularity fallback list (from train)
popular_items = (
    fit_df.groupby("product_id")
    .size()
    .sort_values(ascending=False)
    .index
    .tolist()
)

def rec_cf_only(user_id, k):
    recs = cf_holdout.get_recommendations(int(user_id), top_n=k, exclude_purchased=True)
    return [int(pid) for pid, _ in recs]

def rec_cf_with_pop_fallback(user_id, k):
    rec_items = rec_cf_only(user_id, k)
    if len(rec_items) > 0:
        return rec_items[:k]
    return popular_items[:k]

def eval_served_metrics(test_df, rec_fn, k=10, max_users=5000, seed=42):
    rng_local = np.random.RandomState(seed)
    user_to_items = test_df.groupby("user_id")["product_id"].apply(set)
    users = np.array(list(user_to_items.index))
    if len(users) > max_users:
        users = rng_local.choice(users, size=max_users, replace=False)

    served = 0
    hits = 0
    precisions = []
    recalls = []

    for u in users:
        gt = user_to_items[u]
        rec_items = rec_fn(u, k)[:k]

        if len(rec_items) > 0:
            served += 1

        if not rec_items:
            precisions.append(0.0)
            recalls.append(0.0)
            continue

        inter = len(set(rec_items) & gt)
        if inter > 0:
            hits += 1
        precisions.append(inter / max(1, k))
        recalls.append(inter / max(1, len(gt)))

    n = len(users)
    return {
        "users_evaluated": int(n),
        "served_coverage": served / n if n else 0.0,  # list-level coverage
        "hit_rate@k": hits / n if n else 0.0,
        "precision@k": float(np.mean(precisions)) if precisions else 0.0,
        "recall@k": float(np.mean(recalls)) if recalls else 0.0,
    }

res_cf_only = eval_served_metrics(test_holdout, rec_cf_only, k=K, max_users=MAX_EVAL_USERS, seed=SEED)
res_cf_fb = eval_served_metrics(test_holdout, rec_cf_with_pop_fallback, k=K, max_users=MAX_EVAL_USERS, seed=SEED)

print("\nCF-only served metrics:", res_cf_only)
print("CF + popular fallback served metrics:", res_cf_fb)


INFO:src.models.collaborative_filtering:Fitting Collaborative Filtering Model (ALS)...
INFO:src.models.collaborative_filtering:Creating user-item sparse matrix...


Train rows: 9668502 Test rows: 2415234
Train users: 129905 Test users: 32476
User overlap: 0.0


INFO:src.models.collaborative_filtering:Matrix shape: (129905, 35922)
INFO:src.models.collaborative_filtering:Sparsity: 99.7928%
INFO:src.models.collaborative_filtering:Training ALS model...


  0%|          | 0/10 [00:00<?, ?it/s]

INFO:src.models.collaborative_filtering:ALS training complete
INFO:src.models.collaborative_filtering:ALS factor health: user_finite=100.0000% item_finite=100.0000% | user_nonfinite_rows=0.0000% item_nonfinite_rows=0.0000%
INFO:src.models.collaborative_filtering:ALS check: matrix n_users=129905, n_items=35922 | model.user_factors=129905, model.item_factors=35922
INFO:src.models.collaborative_filtering:Evaluating model...
INFO:src.models.collaborative_filtering:Eval diagnostics: cold_start_user_rate=100.00%, cold_start_item_rate=0.00%, in_matrix_rate=0.00%
INFO:src.models.collaborative_filtering:Eval diagnostics: dropped_empty_items=0
INFO:src.models.collaborative_filtering:RMSE: nan | MAE: nan | Corr: 0.0000 | coverage(all): 0.00% | coverage(in-matrix): 0.00%
INFO:src.models.collaborative_filtering:Predict status counts: {} | fallback_rate_in_matrix=0.00%


CF pair-level metrics: {'rmse': nan, 'mae': nan, 'correlation': 0.0, 'coverage': 0.0, 'coverage_in_matrix': 0.0, 'cold_start_user_rate': 1.0, 'cold_start_item_rate': 0.0}



CF-only served metrics: {'users_evaluated': 5000, 'served_coverage': 0.0, 'hit_rate@k': 0.0, 'precision@k': 0.0, 'recall@k': 0.0}
CF + popular fallback served metrics: {'users_evaluated': 5000, 'served_coverage': 1.0, 'hit_rate@k': 0.8482, 'precision@k': 0.2815, 'recall@k': 0.04633050686068931}
